In [ ]:
import re
import csv
import os

: 

In [ ]:
def sql_to_csv(sql_file_path):
    """
    Chuyển đổi dữ liệu từ tệp SQL có các câu lệnh INSERT INTO thành các tệp CSV riêng biệt cho mỗi bảng.

    Args:
        sql_file_path (str): Đường dẫn đến tệp .sql.
    """
    try:
        with open(sql_file_path, 'r', encoding='utf-8') as f:
            sql_content = f.read()
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy tệp tin '{sql_file_path}'")
        return

    # Biểu thức chính quy để tìm các câu lệnh INSERT INTO và trích xuất tên bảng, cột và giá trị
    insert_pattern = re.compile(r"INSERT INTO \[dbo\]\.\[(.*?)\] \((.*?)\) VALUES\s*([\s\S]*?)(?=GO|;)", re.IGNORECASE)

    matches = insert_pattern.finditer(sql_content)

    tables_data = {}

    for match in matches:
        table_name, columns_str, values_str = match.groups()
        columns = [col.strip().strip('[]') for col in columns_str.split(',')]

        if table_name not in tables_data:
            tables_data[table_name] = {'columns': columns, 'rows': []}

        # Biểu thức chính quy để trích xuất các bộ giá trị từ mệnh đề VALUES
        value_tuples = re.findall(r"\((.*?)\)", values_str)

        for tup in value_tuples:
            try:
                # Sử dụng csv.reader để xử lý các giá trị có dấu nháy
                reader = csv.reader([tup], quotechar="'", escapechar="\\")
                values = next(reader)
                # Làm sạch giá trị: loại bỏ khoảng trắng thừa và tiền tố N'
                cleaned_values = [v.strip().lstrip("N'").rstrip("'") for v in values]
                tables_data[table_name]['rows'].append(cleaned_values)
            except Exception as e:
                print(f"Không thể phân tích cú pháp hàng: {tup} cho bảng {table_name}. Lỗi: {e}")

    # Tạo thư mục để lưu các tệp CSV
    output_dir = 'csv_files'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Ghi dữ liệu vào các tệp CSV
    for table_name, data in tables_data.items():
        file_path = os.path.join(output_dir, f"{table_name}.csv")
        with open(file_path, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(data['columns'])
            writer.writerows(data['rows'])
        print(f"Đã tạo tệp: {file_path}")

# Sử dụng hàm
if __name__ == "__main__":
    sql_file = "PronaFlow_New_Sample_Data_EN_split_by_lines.sql"
    sql_to_csv(sql_file)